In [29]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [30]:
import torch.nn as nn
import torch
import matplotlib.pyplot as plt 
import pandas as pd

from sklearn.datasets import load_iris
import numpy as np
import os, pickle

In [31]:
torch.manual_seed(2)

In [32]:
def load_features(file_dir):
    features = []
    with open(file_dir.replace('.json', '_39.file'), 'rb') as f:
        while True:
            try:
                features.append(pickle.load(f))
            except EOFError:
                break
    # added @chandra
    if not(len(features)):
        print(f"********Warning: Feature array empty for filed_dir = {file_dir.split('/')[-1]}***********")
        return np.zeros(shape=(0,39), dtype=np.float32)
    # @chandra addition ends
    features = np.concatenate(features, axis=0)
    return features

data = []
json_folder = '/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/MCV_accent/jsons/'
jsons = [f.name for f in os.scandir(json_folder) if 'json' in f.name]

X, Y = [], []
for i, accent in enumerate(jsons):
    print("loading", accent, end=' ')
    # added @chandra
    if not os.path.exists(json_folder + accent.replace('.json', '_39.file')):
        print(f"****Warning: {accent}_39.file not present*****")
        continue
    # @chandra addition ends
    json_path = json_folder + accent
    features = load_features(json_path)
    X.append(features)
    Y.append([i]*features.shape[0])
    print(features.shape)
X = np.concatenate(X, axis=0)
Y = np.concatenate(Y, axis=0)
print(X.shape, Y.shape)
print(X.dtype, Y.dtype)

loading philippines.json (3217, 39)
loading unlabelled.json ****Warning: unlabelled.json_39.file not present*****
loading other.json ****Warning: other.json_39.file not present*****
loading wales.json (452, 39)
loading scotland.json (12580, 39)
loading hongkong.json (1613, 39)
loading malaysia.json (1057, 39)
loading indian.json (59797, 39)
loading australia.json (31943, 39)
loading us.json ****Warning: us.json_39.file not present*****
loading southatlandtic.json ********Warning: Feature array empty for filed_dir = southatlandtic.json***********
(0, 39)
loading england.json ****Warning: england.json_39.file not present*****
loading canada.json ********Warning: Feature array empty for filed_dir = canada.json***********
(0, 39)
loading ireland.json ********Warning: Feature array empty for filed_dir = ireland.json***********
(0, 39)
loading newzealand.json ********Warning: Feature array empty for filed_dir = newzealand.json***********
(0, 39)
loading bermuda.json ********Warning: Feature 

In [33]:
Y[:4]

array([0., 0., 0., 0.])

In [34]:
Y[-150]

8.0

In [35]:
from torch.utils.data import Dataset, DataLoader
class Data(Dataset):
    def __init__(self, X, Y):
        self.x = X
        self.y = Y
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [36]:
train_dataset = Data(X, Y)
trainloader=DataLoader(dataset=train_dataset,batch_size=64)

In [37]:
class Net(nn.Module):
    def __init__(self,D_in,H,D_out):
        super(Net,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.linear2=nn.Linear(H,D_out)
    
    def forward(self,x):
#         print(x.shape)
        x=torch.sigmoid(self.linear1(x))  
        x=self.linear2(x)
        return x

In [38]:
input_dim = 39     # how many Variables are in the dataset
hidden_dim = 25 # hidden layers
# changed @chandra
# output_dim = 7    # number of classes
output_dim = len(jsons)
# change ends @chandra
model=Net(input_dim,hidden_dim,output_dim)

print('W:',list(model.parameters())[0].size())
print('b',list(model.parameters())[1].size())

W: torch.Size([25, 39])
b torch.Size([25])


In [39]:
criterion=nn.CrossEntropyLoss()

In [40]:
learning_rate=0.1
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

In [41]:
n_epochs=1000
loss_list=[]

for epoch in range(n_epochs):
    print(f"epoch: {epoch}")
    model.train()
    for x, y in trainloader:
        optimizer.zero_grad()
        z=model(x)
        loss=criterion(z,y.type(torch.LongTensor))
        loss.backward()
        optimizer.step()
        loss_list.append(loss.data)

epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18
epoch: 19
epoch: 20
epoch: 21
epoch: 22
epoch: 23
epoch: 24
epoch: 25
epoch: 26
epoch: 27
epoch: 28
epoch: 29
epoch: 30
epoch: 31
epoch: 32
epoch: 33
epoch: 34


KeyboardInterrupt: 

In [13]:
z[0:10]

tensor([[-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853]],
       grad_fn=<SliceBackward0>)

In [14]:
z.data

tensor([[-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.2570,  0.3227,  5.6853],
        [-1.3262, -1.1447, -0.9302, -0.6505, -0.257

In [50]:
z=model(x_val)

In [17]:
print(X.shape)
print(z.shape)

(26967, 39)
torch.Size([23, 7])


In [52]:
yhat=torch.max(z.data,1)
yhat

torch.return_types.max(
values=tensor([3.1840, 8.2315, 7.8008, 2.9988, 3.3571, 7.9788, 3.8208, 3.7191, 2.5231,
        3.7578, 3.1040, 7.7698, 8.3856, 7.7692, 8.3334, 3.2011, 6.4714, 3.6179,
        2.9457, 6.4669, 7.6269, 2.9646, 7.7773, 6.2602, 3.6186, 4.7797, 6.2235,
        6.2391, 7.6216, 7.5300, 8.5784, 8.7336, 3.9174, 7.8728, 7.9532, 4.7847,
        3.5662, 8.1570, 8.3068, 8.6275, 5.4449, 3.2648, 3.5053, 8.5569, 8.3481,
        3.7902, 2.7955, 4.4667, 3.7982, 5.9606]),
indices=tensor([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2,
        2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2,
        1, 2]))

In [53]:
z.data.argmax(axis=1)

tensor([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2,
        2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2,
        1, 2])

In [54]:
y_val

tensor([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2,
        2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2,
        1, 2])

In [55]:
val_accuracy = sum(z.data.argmax(axis=1)==y_val)/len(y_val)
val_accuracy

tensor(1.)

In [23]:
import json

def load_features(file_dir):
    features = []
    with open(file_dir.replace('.json', '_39.file'), 'rb') as f:
        while True:
            try:
                features.append(pickle.load(f))
            except EOFError:
                break
    features = np.concatenate(features, axis=0)
    return features

data = []
json_folder = '/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/MCV_accent/jsons/'
jsons = [f.name for f in os.scandir(json_folder) if 'json' in f.name and f.name.split('.')[0] not in ['unlabelled', 'other']]

json2label = dict(zip(jsons, range(len(jsons))))
sample2feature = {}
for i, accent in enumerate(jsons):
    try:
        print("loading", accent, end=' ')
        json_path = json_folder + accent
        json_file = open(json_path)
        json_item_list = [line for line in json_file]
        json_item_list = json_item_list[:1000]
        json_item_list = [json.loads(line.strip()) for line in json_item_list]
        features = load_features(json_path)
        sample_names = [i['audio_filepath'].split('/')[-1][:-4] for i in json_item_list]
        for i, name in enumerate(sample_names):
            sample2feature[name] = features[i]
    except:
        continue

loading philippines.json loading wales.json loading scotland.json loading hongkong.json loading malaysia.json loading indian.json loading australia.json loading us.json loading southatlandtic.json loading england.json loading canada.json loading ireland.json loading newzealand.json loading bermuda.json loading singapore.json loading african.json 

In [29]:
import csv

X_train, X_test, Y_train, Y_test = [], [], [], []
with open('data/with_all_1k/train.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile, delimiter="\t")
    next(csvreader)
    for row in csvreader:
        try:
#             print(row[0], type(row[0]))
            if row[0] in sample2feature:
                X_train.append(sample2feature[row[0]])
                Y_train.append(json2label[row[-1]])
            else:
                X_test.append(sample2feature[row[0]])
                Y_test.append(json2label[row[-1]])
        except:
            continue
            
with open('data/with_all_1k/test.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile, delimiter="\t")
    next(csvreader)
    for row in csvreader:
        try:
            if row[0] in sample2feature:
                X_train.append(sample2feature[row[0]])
                Y_train.append(json2label[row[-1]])
            else:
                X_test.append(sample2feature[row[0]])
                Y_test.append(json2label[row[-1]])
        except:
            continue

# X_train, X_test, Y_train, Y_test = np.concatenate(X_train), np.concatenate(X_test), np.concatenate(Y_train), np.concatenate(Y_test)

In [25]:
sample2feature.keys()

dict_keys(['common_voice_en_18730133', 'common_voice_en_18730135', 'common_voice_en_18730136', 'common_voice_en_18730137', 'common_voice_en_18730154', 'common_voice_en_21087880', 'common_voice_en_21087881', 'common_voice_en_21087883', 'common_voice_en_21087884', 'common_voice_en_21087886', 'common_voice_en_20015753', 'common_voice_en_20015754', 'common_voice_en_20015755', 'common_voice_en_20015756', 'common_voice_en_20015757', 'common_voice_en_20311852', 'common_voice_en_20311853', 'common_voice_en_20311856', 'common_voice_en_20311861', 'common_voice_en_20311864', 'common_voice_en_20171460', 'common_voice_en_20171461', 'common_voice_en_20171462', 'common_voice_en_20171463', 'common_voice_en_20171464', 'common_voice_en_20837535', 'common_voice_en_20837536', 'common_voice_en_20837537', 'common_voice_en_20837538', 'common_voice_en_20837540', 'common_voice_en_26947543', 'common_voice_en_505390', 'common_voice_en_488440', 'common_voice_en_488441', 'common_voice_en_488443', 'common_voice_en_

In [28]:
'common_voice_en_18730133' in sample2feature

True